In [1]:
import sys,tweepy,csv,re
from textblob import TextBlob
import matplotlib.pyplot as plt
import tweepy

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)


class SentimentAnalysis:

    def __init__(self):
        self.tweets = []
        self.tweetText = []

    def DownloadData(self):
        
        # Authentification API Twitter
        
        consumerKey = ''
        consumerSecret = ''
        accessToken = ''
        accessTokenSecret = ''
        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth)

        # Saisie du terme à rechercher et du nombre de tweets à récupérer
        
        searchTerm = input("Enter Keyword/Tag to search about: ")
        NoOfTerms = int(input("Enter how many tweets to search: "))

        # Recherche des tweets
        self.tweets = tweepy.Cursor(api.search, q=searchTerm, since="2018-07-13", until = "2018-07-18", lang = "en").items(NoOfTerms)

        # Ouverture d'un fichier pour stocker les données
        csvFile = open('Tweets.csv', 'a', newline = '')

        # Ecriture dans le fichier csv
        csvWriter = csv.writer(csvFile)


        # Variables sentiments 
        
        polarity = 0
        positive = 0
        wpositive = 0
        spositive = 0
        negative = 0
        wnegative = 0
        snegative = 0
        neutral = 0


        # Boucle sur les tweets à analyser
         
        for tweet in self.tweets:

            self.tweetText.append(self.cleanTweet(tweet.text).encode('utf-8'))
            #print (tweet.text.translate(non_bmp_map)) 
            
            print (self.cleanTweet(tweet.text))
            
            analysis = TextBlob(self.cleanTweet(tweet.text))
            print(analysis.sentiment)  # Affiche la polarité du tweet
            
            polarity += analysis.sentiment.polarity  # Ajoute la polarité pour effectuer une moyenne (plus bas)
            #print(polarity)

            if (analysis.sentiment.polarity == 0): 
                neutral += 1
            elif (analysis.sentiment.polarity > 0 and analysis.sentiment.polarity <= 0.3):
                wpositive += 1
            elif (analysis.sentiment.polarity > 0.3 and analysis.sentiment.polarity <= 0.6):
                positive += 1
            elif (analysis.sentiment.polarity > 0.6 and analysis.sentiment.polarity <= 1):
                spositive += 1
            elif (analysis.sentiment.polarity > -0.3 and analysis.sentiment.polarity <= 0):
                wnegative += 1
            elif (analysis.sentiment.polarity > -0.6 and analysis.sentiment.polarity <= -0.3):
                negative += 1
            elif (analysis.sentiment.polarity > -1 and analysis.sentiment.polarity <= -0.6):
                snegative += 1
            #Ecriture des tweets dans un fichier csv (Date + texte)
            csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
        
        #Fermeture du fichier csv
        csvFile.close()

        # Calcul des % pour chaque tendance
        positive = self.percentage(positive, NoOfTerms)
        wpositive = self.percentage(wpositive, NoOfTerms)
        spositive = self.percentage(spositive, NoOfTerms)
        negative = self.percentage(negative, NoOfTerms)
        wnegative = self.percentage(wnegative, NoOfTerms)
        snegative = self.percentage(snegative, NoOfTerms)
        neutral = self.percentage(neutral, NoOfTerms)

        # Calcul de la polarité moyenne
        
        polarity = polarity / NoOfTerms
        
        # Affichage des datas
        print("How people are reacting on " + searchTerm + " by analyzing " + str(NoOfTerms) + " tweets.")
        print()
        print("General Report: ")

        if (polarity == 0):
            print("Neutral")
        elif (polarity > 0 and polarity <= 0.3):
            print("Weakly Positive")
        elif (polarity > 0.3 and polarity <= 0.6):
            print("Positive")
        elif (polarity > 0.6 and polarity <= 1):
            print("Strongly Positive")
        elif (polarity > -0.3 and polarity <= 0):
            print("Weakly Negative")
        elif (polarity > -0.6 and polarity <= -0.3):
            print("Negative")
        elif (polarity > -1 and polarity <= -0.6):
            print("Strongly Negative")

        print()
        print("Detailed Report: ")
        print(str(positive) + "% people thought it was positive")
        print(str(wpositive) + "% people thought it was weakly positive")
        print(str(spositive) + "% people thought it was strongly positive")
        print(str(negative) + "% people thought it was negative")
        print(str(wnegative) + "% people thought it was weakly negative")
        print(str(snegative) + "% people thought it was strongly negative")
        print(str(neutral) + "% people thought it was neutral")

        self.plotPieChart(positive, wpositive, spositive, negative, wnegative, snegative, neutral, searchTerm, NoOfTerms)


    def cleanTweet(self, tweet):
        
        # Supression des caractères spéciaux du tweet
        
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w +:\ / \ / \S +)", " ", tweet).split())
    
        #Convert to lower case
        #tweet = tweet.lower()
        #Convert www.* or https?://* to URL
        #tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
        #Convert @username to AT_USER
        #tweet = re.sub('@[^\s]+','AT_USER',tweet)
        #Remove additional white spaces
        #tweet = re.sub('[\s]+', ' ', tweet)
        #Replace #word with word
        #tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
        #trim
        #tweet = tweet.strip('\'"')
        #return tweet 
        
    #Fonction calcul du pourcentage
    
    def percentage(self, part, whole):
        temp = 100 * float(part) / float(whole)
        return format(temp, '.2f')

    #Fonction d'affichage graphique pour visualiser l'analyse sentimentale.
    
    def plotPieChart(self, positive, wpositive, spositive, negative, wnegative, snegative, neutral, searchTerm, noOfSearchTerms):
        labels = ['Positive [' + str(positive) + '%]', 'Weakly Positive [' + str(wpositive) + '%]','Strongly Positive [' + str(spositive) + '%]', 'Neutral [' + str(neutral) + '%]',
                  'Negative [' + str(negative) + '%]', 'Weakly Negative [' + str(wnegative) + '%]', 'Strongly Negative [' + str(snegative) + '%]']
        sizes = [positive, wpositive, spositive, neutral, negative, wnegative, snegative]
        colors = ['yellowgreen','lightgreen','darkgreen', 'gold', 'red','lightsalmon','darkred']
        patches, texts = plt.pie(sizes, colors=colors, startangle=90)
        plt.legend(patches, labels, loc="best")
        plt.title('How people are reacting on ' + searchTerm + ' by analyzing ' + str(noOfSearchTerms) + ' Tweets.')
        plt.axis('equal')
        plt.tight_layout()
        plt.show()



if __name__== "__main__":
    sa = SentimentAnalysis()
    sa.DownloadData()



Enter Keyword/Tag to search about: #France
Enter how many tweets to search: 200
Normandie #Normandy Britain #France https://t.co/ZBC54MqfF1
Sentiment(polarity=0.0, subjectivity=0.0)
RT Are the citizens of #France aware they actually won the #WorldCupFinal2018 ? What would have gone down had they lost?…
Sentiment(polarity=0.03148148148148147, subjectivity=0.21296296296296294)
RT May #racism #xenophobia and #Islamophobia be erased from society 7 #Muslims from different backgrounds all played for #F…
Sentiment(polarity=0.0, subjectivity=0.6)
A bonkers World Cup is over &amp the team we rated the highest before the tourney won it France How did our model https://t.co/tOeyv5v4kO
Sentiment(polarity=0.0, subjectivity=0.0)
RT #CRO heroes are home and ZAGREB is BURNING True #champions! #France won the #WorldCup but #Croatia 🇭 won the #heart…
Sentiment(polarity=0.4375, subjectivity=0.65)
RT Wednesday Afternoon Hello Twitterland 💕💙 Let's visit a castle in #France 🇫 https://t.co/dOa6Rz0v2x
Sentime

RT The main finale is here #FRA Vs #CRO We've seen some amazing matches till now but only one team can win the #WorldCup Who wi…
Sentiment(polarity=0.3916666666666667, subjectivity=0.6583333333333333)
RT Empire State Building lit up for #WorldCup18 winners #France🇫 https://t.co/yYBZ7ii81r
Sentiment(polarity=0.0, subjectivity=0.0)
#France’s young #WorldCup-winning #football team have staged a victorious homecoming parade in an open-top bus dow https://t.co/WJZMqOVV5s
Sentiment(polarity=-0.075, subjectivity=0.31666666666666665)
RT What an Amazing Deal!! Click The LINK BELOW! Before You Miss the SALE #worldcup #worldcup2018 #france #croatia #socc…
Sentiment(polarity=1.0, subjectivity=0.9)
RT __ https://t.co/OPAzVz3Xes discount on hotel rooms NOLA Jazz House from USD 30 in #New Orleans #Louisiana /#Family #holi…
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
RT We are always arriving and departing and We find the a new way of looking at things #Chamonix #France h

RT The planned #France rollout of #5G may be ambitious by European terms but far behind the #UnitedStates #SouthKorea &amp #Chi…
Sentiment(polarity=-0.012500000000000011, subjectivity=0.6125)
RT Hello dear DACC lovers and users As the Would Cup reaches the final stage here comes the last World Cup Bounty Campa…
Sentiment(polarity=0.0, subjectivity=0.5333333333333333)
Leaving Rouen France tonight Sailing all night I need to get to sleep 1am #seine #seineriver #travelling https://t.co/dP0VnBBCzw
Sentiment(polarity=0.0, subjectivity=0.0)
RT Am I the only one who saw this woman steal a medal? 👀😂🤦🏿‍♂ Did she think we wouldn’t see What don’t I know #World…
Sentiment(polarity=0.0, subjectivity=1.0)
RT Shut down #Iran 's embassies house of terrorism &amp #ExpellraDiplomatTerrorists #UK #Germnay #belgium #France #Europe…
Sentiment(polarity=-0.15555555555555559, subjectivity=0.2888888888888889)
Dude sent an PVP challenge to me then took it back #LOL #HeroHunters #UK #France #Germany #Australia #

RT Eiffel Tower Paris Footprint https://t.co/ClGxZQFZay #eiffeltower #Paris #France #Europe #travelphotography #Travel …
Sentiment(polarity=0.0, subjectivity=0.0)
RT Huge congratulations to you President to the people of #France and a round of applause to your brillia…
Sentiment(polarity=0.10000000000000003, subjectivity=0.65)
RT Putin is enjoying himself at the back #worldcup #croatia #france https://t.co/GjKPOCBo7g
Sentiment(polarity=0.25, subjectivity=0.3)
RT Come meet #NewYorkCoin #cryptocurrency in Paris #France Wednesday July 18th 2018 #NYCoin Conference Info here htt…
Sentiment(polarity=0.0, subjectivity=0.0)
RT #Croatia lost the game #France lost the future. #CRO #FRA #WorldCupFinal https://t.co/sOiXsQo0t5
Sentiment(polarity=-0.2, subjectivity=0.2625)
RT 🇫 Congratulations to #France on winning the 2018 #WorldCup 🌟🌟
Sentiment(polarity=0.5, subjectivity=0.75)
Day &amp Night views for Mondial #2018 #russia #congratulation #france https://t.co/T1nA3gWode
Sentiment(polarity=0.0, sub

<Figure size 640x480 with 1 Axes>